In [ ]:
# Pip install
!pip install feedparser
!pip install newspaper3k
!pip install konlpy

In [ ]:
# IMPORT LIB & URLS
import numpy as np
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter
from operator import eq

urls = ("http://rss.etnews.com/Section901.xml", "http://rss.etnews.com/Section902.xml","http://rss.etnews.com/Section903.xml",None)

In [ ]:
def fileload():
  f = open("./korea.txt", 'r', encoding='utf-8')
  stopwords = []
  for line in f.readlines():
    stopwords.append(line.rstrip())
  f.close()
  # stop_words = set(stopwords.words('korea'))
  #print(stopwords)
  return stopwords

In [ ]:
# FUNCTION (1) : 기사본문 중 50개의 단어 품사별로
def get_tags(text,ntags=50):
  num_unique_words=0
  num_most_freq=[]
  word_name=[]
  stopwords = fileload()

  spliter=Okt() # SPLIT -> 품사별로 나눠주는 오퍼레이터
  nouns = spliter.nouns(text)
  delete=[]
  # result=[word for word in word_name if not word in stopwords]
  for w in nouns: 
    if w not in stopwords: 
      delete.append(w) 
  count = Counter(delete) # 명사와 나오는 횟수 리턴해줌
  return_list = []

  for n,c in count.most_common(ntags) :#most_common : count를 sorting 해줌
    num_unique_words += 1
    word_name.append(n)
    num_most_freq.append(c)
  return word_name, num_unique_words, num_most_freq# 리턴 값 = 단어 수, 빈도, 전체 # return num_unique_words, num_most_freq, return_list

In [ ]:
# FUNCTION (3) : urls의 제목과 링크를 어펜드 & 리턴
def crawl_rss(urls):
  arr_rss = []
  for url in urls:
    #print("Success Crawl",url)
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries: #entries
      arr_rss.append({'title':p.title, 'link':p.link}) # descriptions X : 일부만 보여주기 때문에, 링크 타고가서 전부 읽어와야 => 타이틀, 링크만 가져온다
  return arr_rss

In [ ]:
# FUNCTION (4) : url 한개 들어오면, 다운로드 & 파싱 => 제목이랑 내용 리턴
def crawl_article(url, language='ko'):
  #print("Success Article",url)
  var_article = Article(url, language = language)
  var_article.download()
  var_article.parse()
  return var_article.title, var_article.text

In [ ]:
spliter = Okt() 
article_list = crawl_rss(urls)
#print(article_list)
clean_text = []
for article in article_list:
  _, text = crawl_article(article['link'])
  article['text'] = text

In [ ]:
# MAIN : 실습(2) : 많이 나온 단어 순서대로 출력하기
word_list = []
num_list = []
result ={}
temp = {}

# Text Parsing
print('[Parsing Text]')
for article in article_list:
  word_name, num_unique_words, num_most_freq= get_tags(article['text'], 10000)
  word_list.extend(word_name)
  num_list.extend(num_most_freq)

# 중복제거
for key_check in set(word_list):
  tmp = 0
  for key, value in zip(word_list,num_list):
    if key_check == key:
      tmp += value
  result[key_check] = tmp

# 정렬
result = sorted(result.items(), key = lambda x : x[1], reverse = True)

In [ ]:
print(result)